#Building an QA System for a Healthcare Platform
The code provides a structured way to compare the performance of a small model (BERT) and a large model (T5) in the context of healthcare-related QA tasks. By analyzing the answers from both models, the healthcare startup can make a well-informed decision about which model to deploy based on their specific needs (e.g., speed vs. accuracy).

## Section 1: Package Installation and Define Dataset


In [ ]:
# Install necessary libraries: PyTorch and Hugging Face transformers
# - torch: PyTorch is the deep learning framework required to run pre-trained neural network models.
# - transformers: Hugging Face's library that provides ready-to-use NLP models like BERT and T5.
!pip install torch transformers

# Import the time module to measure the speed of model responses.
import time

# Import the Hugging Face pipeline API.
# - pipeline provides a high-level interface for loading and using pre-trained models
#   without needing to handle the low-level details (tokenization, model forward pass, etc.).
from transformers import pipeline

# Define the context (knowledge source) containing information about multiple diseases.
# - This simulates the knowledge base that the QA models will reference.
# - In a real-world healthcare startup, the context could come from medical databases,
#   patient records, or curated clinical guidelines.
context = ("COVID-19 is a disease caused by the SARS-CoV-2 virus, which primarily affects the respiratory system. Symptoms can range from mild, such as fever and cough, to severe, such as difficulty breathing and pneumonia. Vaccines have been developed to help prevent the spread of the virus."
           "Cancer is a group of diseases involving abnormal cell growth with the potential to invade or spread to other parts of the body. Common treatments include surgery, chemotherapy, and radiation therapy."
           "Diabetes is a chronic disease that occurs when the pancreas does not produce enough insulin or when the body cannot effectively use the insulin it produces. Type 1 diabetes is typically managed with insulin injections, while Type 2 diabetes can often be managed with diet, exercise, and oral medications."
           "HIV (Human Immunodeficiency Virus) weakens the immune system by destroying important cells that fight disease and infection. Without treatment, HIV can lead to the disease AIDS (Acquired Immunodeficiency Syndrome). Antiretroviral therapy (ART) is the treatment for HIV and can control the virus, helping people with HIV live longer, healthier lives."
)

# Define a set of disease-related questions that will be asked to both models.
# - These represent realistic healthcare queries a patient or doctor might ask.
# - The answers will help compare how well each model understands the context.
questions = [
    "What causes COVID-19?",           # Tests if the model identifies SARS-CoV-2 as the cause.
    "How is cancer treated?",          # Tests if the model retrieves surgery, chemo, radiation.
    "How is Type 1 diabetes managed?", # Tests if the model mentions insulin injections.
    "What is the treatment for HIV?"   # Tests if the model mentions ART (antiretroviral therapy).
]



## Section 2: Load the BERT Model and Tokenizer


In [ ]:
# We use the BERT model trained on the SQuAD2 dataset for QA tasks.
from transformers import BertForQuestionAnswering, AutoTokenizer

# Load the BERT model for question answering.
bert_model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')

# Load the tokenizer that will convert text into the format that the BERT model understands.
bert_tokenizer = AutoTokenizer.from_pretrained('deepset/bert-base-cased-squad2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]


## Section 3: Load the T5 Model and Tokenizer


In [ ]:
# T5 is another pre-trained model used for various NLP tasks, including question answering.
# Unlike BERT, T5 is an encoder-decoder model and expects different input formatting.
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the T5 model (large version) for question answering.
t5_model = T5ForConditionalGeneration.from_pretrained('t5-large')

# Load the T5 tokenizer which encodes questions and context in a format T5 can process.
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565



## Section 4: Define Helper Functions for Both Models


In [ ]:
# Function for BERT-based question answering.
# This function takes a question and context, uses the BERT model, and returns an answer and the time taken.
def bert_qa(question, context):
    # Initialize the pipeline for question-answering with BERT.
    nlp = pipeline('question-answering', model=bert_model, tokenizer=bert_tokenizer)

    # Measure the start time.
    start_time = time.time()

    # Get the answer from BERT by providing the question and context.
    result = nlp({'question': question, 'context': context})

    # Measure the end time.
    end_time = time.time()

    # Calculate the time taken.
    bert_time = end_time - start_time

    # Return the answer and time taken by BERT.
    return result['answer'], bert_time

# Function for T5-based question answering.
# This function takes a question and context, formats the input for T5, and returns an answer and the time taken.
def t5_qa(question, context):
    # Format the input as required by T5: "question: <question> context: <context>"
    input_text = f"question: {question} context: {context}"

    # Measure the start time.
    start_time = time.time()

    # Encode the input text into token IDs for the model.
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt')

    # Generate the answer using the T5 model.
    outputs = t5_model.generate(input_ids)

    # Measure the end time.
    end_time = time.time()

    # Calculate the time taken.
    t5_time = end_time - start_time

    # Decode the generated tokens into a readable answer.
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Return the answer and time taken by T5.
    return answer, t5_time

## Section 5: Compare Answers and Speed from BERT and T5

In [ ]:
# In this section, we compare the answers provided by BERT and T5 for each question, along with the time taken by each.

def compare_qa_models(questions, context):
    results = []  # Initialize a list to store the results.

    # Loop over all the questions to test both models.
    for question in questions:
        # Get the answer and time from BERT for the current question.
        bert_answer, bert_time = bert_qa(question, context)

        # Get the answer and time from T5 for the same question.
        t5_answer, t5_time = t5_qa(question, context)

        # Store the results including answers and time taken by both models.
        results.append({
            'Question': question,
            'BERT Answer': bert_answer,
            'BERT Time (s)': bert_time,
            'T5 Answer': t5_answer,
            'T5 Time (s)': t5_time
        })

    return results  # Return the list of results for each question.

# Call the comparison function to test both models on the defined disease-related questions and context.
results = compare_qa_models(questions, context)

# Convert the results into a DataFrame for better visualization.
import pandas as pd
df_results = pd.DataFrame(results)

# Display the DataFrame using pandas' built-in display method.
print("Disease-Related QA Model Comparison Results (with Speed)")
print(df_results)


Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu


Disease-Related QA Model Comparison Results (with Speed)
                          Question  \
0            What causes COVID-19?   
1           How is cancer treated?   
2  How is Type 1 diabetes managed?   
3   What is the treatment for HIV?   

                                    BERT Answer  BERT Time (s)  \
0                              SARS-CoV-2 virus       2.979182   
1  surgery, chemotherapy, and radiation therapy       0.947953   
2                            insulin injections       0.932490   
3                        Antiretroviral therapy       0.936283   

                                      T5 Answer  T5 Time (s)  
0                              SARS-CoV-2 virus    20.836130  
1  surgery, chemotherapy, and radiation therapy     6.547412  
2                            insulin injections     4.760073  
3                        Antiretroviral therapy     6.258555  
